In [ ]:
# default_exp core.transfer_model

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Transfer learning temporal model 
> Trains the temporal model from the Video Pose3D checkpoint made for inference. 

In [ ]:
#export
import os
import time

import matplotlib
import numpy as np
import torch

matplotlib.use('Agg')
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn

from runningpose.core.generators import ChunkedGenerator, UnchunkedGenerator
from runningpose.core.loss import mpjpe
from runningpose.core.model import TemporalModel
from runningpose.core.runningpose_dataset import RunningposeDataset
from runningpose.core.camera import normalize_screen_coordinates


### Load
#### Training data

In [ ]:
#export
print('Loading training dataset...')
keypoints_3D = np.load('data_3d_train.npz', allow_pickle=True)
keypoints_3D = keypoints_3D['positions_3d'].item()
joints_left = [1, 3, 5, 7, 10, 11, 14]
joints_right = [0, 2, 4, 6, 8, 9, 13]
print(keypoints_3D.keys())
print(keypoints_3D['Ioanna3_Camera1_170Hz_3D_keypoints'].shape)

Loading training dataset...
dict_keys(['Ioanna1_Camera1_170Hz_3D_keypoints', 'Ioanna2_Camera1_170Hz_3D_keypoints', 'Ioanna3_Camera1_170Hz_3D_keypoints', 'Josef1_Camera1_170Hz_3D_keypoints', 'Josef2_Camera1_170Hz_3D_keypoints', 'Josef3_Camera1_170Hz_3D_keypoints', 'Josef4_Camera1_170Hz_3D_keypoints', 'Ioanna1_Camera2_170Hz_3D_keypoints', 'Ioanna2_Camera2_170Hz_3D_keypoints', 'Ioanna3_Camera2_170Hz_3D_keypoints', 'Josef1_Camera2_170Hz_3D_keypoints', 'Josef2_Camera2_170Hz_3D_keypoints', 'Josef3_Camera2_170Hz_3D_keypoints', 'Josef4_Camera2_170Hz_3D_keypoints', 'Ioanna1_Camera3_170Hz_3D_keypoints', 'Ioanna2_Camera3_170Hz_3D_keypoints', 'Ioanna3_Camera3_170Hz_3D_keypoints', 'Josef1_Camera3_170Hz_3D_keypoints', 'Josef2_Camera3_170Hz_3D_keypoints', 'Josef3_Camera3_170Hz_3D_keypoints', 'Josef4_Camera3_170Hz_3D_keypoints'])
(374, 18, 3)


We can see that we have a dictionary with all the videos and then a 
custom dictionary with a list with an array with (frames, keypoints, dim) 

In [ ]:
#export
print('Loading 2D training detections...')
keypoints_2D = np.load('data_2d_custom_trainingdata.npz', allow_pickle=True)
keypoints_2D_metadata = keypoints_2D['metadata'].item()
keypoints_2D_symmetry = keypoints_2D_metadata['keypoints_symmetry']
kps_left = list(keypoints_2D_symmetry[0])
kps_right = list(keypoints_2D_symmetry[1])
keypoints_2D = keypoints_2D['positions_2d'].item()
keypoints_2D = dict(sorted(keypoints_2D.items()))
print(keypoints_2D.keys())
print(keypoints_2D['miqus1_Ioanna_01.avi']['custom'][0].shape)

Loading 2D training detections...
dict_keys(['miqus1_Ioanna_01.avi', 'miqus1_Ioanna_02.avi', 'miqus1_Ioanna_03.avi', 'miqus1_Josef_01.avi', 'miqus1_Josef_02.avi', 'miqus1_Josef_03.avi', 'miqus1_Josef_04.avi', 'miqus2_Ioanna_01.avi', 'miqus2_Ioanna_02.avi', 'miqus2_Ioanna_03.avi', 'miqus2_Josef_01.avi', 'miqus2_Josef_02.avi', 'miqus2_Josef_03.avi', 'miqus2_Josef_04.avi', 'miqus3_Ioanna_01.avi', 'miqus3_Ioanna_02.avi', 'miqus3_Ioanna_03.avi', 'miqus3_Josef_01.avi', 'miqus3_Josef_02.avi', 'miqus3_Josef_03.avi', 'miqus3_Josef_04.avi'])
(323, 17, 2)


#### Normalize training inputs
The resolution of the third camera is diffrent due to being cropped. 
During our datacollection we got some other people in the shot. Why we had to
crop the video. 

In [ ]:
#export
# Normalize camera frame
subjects = [
    'miqus1_Ioanna_01.avi', 'miqus1_Ioanna_02.avi', 'miqus1_Ioanna_03.avi', 
    'miqus1_Josef_01.avi', 'miqus1_Josef_02.avi', 'miqus1_Josef_03.avi', 
    'miqus1_Josef_04.avi', 'miqus2_Ioanna_01.avi', 'miqus2_Ioanna_02.avi', 
    'miqus2_Ioanna_03.avi', 'miqus2_Josef_01.avi', 'miqus2_Josef_02.avi', 
    'miqus2_Josef_03.avi', 'miqus2_Josef_04.avi'
]
for subject in subjects:
    for action in keypoints_2D[subject]:
        for idx, kps in enumerate(keypoints_2D[subject][action]):
            kps = normalize_screen_coordinates(kps, w=1920, h=1088)
            keypoints_2D[subject][action][idx] = kps

In [ ]:
#export
subjects_extra_cut = ['miqus3_Ioanna_01.avi', 'miqus3_Josef_01.avi']
for subject in subjects_extra_cut:
    for action in keypoints_2D[subject]:
        for idx, kps in enumerate(keypoints_2D[subject][action]):
            kps = normalize_screen_coordinates(kps, w=1350, h=1088)
            keypoints_2D[subject][action][idx] = kps

In [ ]:
#export
subjects_cut = [
    'miqus3_Ioanna_02.avi', 'miqus3_Ioanna_03.avi', 'miqus3_Josef_02.avi', 
    'miqus3_Josef_03.avi', 'miqus3_Josef_04.avi'
]
for subject in subjects_cut:
    for action in keypoints_2D[subject]:
        for idx, kps in enumerate(keypoints_2D[subject][action]):
                kps = normalize_screen_coordinates(kps, w=1480, h=1088)
                keypoints_2D[subject][action][idx] = kps

#### Validation data

In [ ]:
#export
print('Loading validation dataset...')
keypoints_3D_val = np.load('data_3d_val.npz', allow_pickle=True)
keypoints_3D_val = keypoints_3D_val['positions_3d'].item()
print(keypoints_3D_val.keys())
# print(keypoints_3D['Ioanna1_Camera1_170Hz_3D_keypoints'].shape)

Loading validation dataset...
dict_keys(['Tindra1_Camera1_170Hz_3D_keypoints', 'Tindra2_Camera1_170Hz_3D_keypoints', 'Tindra3_Camera1_170Hz_3D_keypoints', 'Tindra1_Camera2_170Hz_3D_keypoints', 'Tindra2_Camera2_170Hz_3D_keypoints', 'Tindra3_Camera2_170Hz_3D_keypoints', 'Tindra1_Camera3_170Hz_3D_keypoints', 'Tindra2_Camera3_170Hz_3D_keypoints', 'Tindra3_Camera3_170Hz_3D_keypoints'])


In [ ]:
#export
print('Loading 2D training detections...')
keypoints_2D_val = np.load('data_2d_custom_validationdata.npz', allow_pickle=True)
keypoints_2D_val = keypoints_2D_val['positions_2d'].item()
keypoints_2D_val = dict(sorted(keypoints_2D_val.items()))
print(keypoints_2D_val.keys())

Loading 2D training detections...
dict_keys(['miqus1_Tindra_01.avi', 'miqus1_Tindra_02.avi', 'miqus1_Tindra_03.avi', 'miqus2_Tindra_01.avi', 'miqus2_Tindra_02.avi', 'miqus2_Tindra_03.avi', 'miqus3_Tindra_01.avi', 'miqus3_Tindra_02.avi', 'miqus3_Tindra_03.avi'])


#### Normalize validation input

In [ ]:
#export
subjects = [
    'miqus1_Tindra_01.avi', 'miqus1_Tindra_02.avi', 'miqus1_Tindra_03.avi', 
    'miqus2_Tindra_01.avi', 'miqus2_Tindra_02.avi', 'miqus2_Tindra_03.avi'
]
for subject in subjects:
    for action in keypoints_2D_val[subject]:
        for idx, kps in enumerate(keypoints_2D_val[subject][action]):
            kps[..., :2] = normalize_screen_coordinates(kps[..., :2], w=1920, h=1088)
            keypoints_2D_val[subject][action][idx] = kps

In [ ]:
#export
subjects_cut = [
    'miqus3_Tindra_01.avi', 'miqus3_Tindra_02.avi', 
    'miqus3_Tindra_03.avi'
]
for subject in subjects_cut:
    for action in keypoints_2D_val[subject]:
        for idx, kps in enumerate(keypoints_2D_val[subject][action]):
            kps[..., :2] = normalize_screen_coordinates(kps[..., :2], w=1480, h=1088)
            keypoints_2D_val[subject][action][idx] = kps

Assert that we have the same number of frames in the 2D and 3D data.

In [ ]:
#export
# Wrong cut .avi should have been 4.42 long but is 4.44 
keypoints_2D['miqus3_Josef_03.avi']['custom'][0] = keypoints_2D['miqus3_Josef_03.avi']['custom'][0][:374]

In [ ]:
#export
shapes_3d = []
for subject in keypoints_3D.keys():
    shapes_3d.append(keypoints_3D[subject].shape)
print(shapes_3d)

[(323, 18, 3), (272, 18, 3), (374, 18, 3), (238, 18, 3), (289, 18, 3), (306, 18, 3), (357, 18, 3), (238, 18, 3), (187, 18, 3), (289, 18, 3), (170, 18, 3), (221, 18, 3), (221, 18, 3), (272, 18, 3), (408, 18, 3), (374, 18, 3), (391, 18, 3), (323, 18, 3), (374, 18, 3), (374, 18, 3), (425, 18, 3)]


In [ ]:
#export
shapes_2d = []
for subject in keypoints_2D.keys():
    shapes_2d.append(keypoints_2D[subject]['custom'][0].shape)
print(shapes_2d)

[(323, 17, 2), (272, 17, 2), (374, 17, 2), (238, 17, 2), (289, 17, 2), (306, 17, 2), (357, 17, 2), (238, 17, 2), (187, 17, 2), (289, 17, 2), (170, 17, 2), (221, 17, 2), (221, 17, 2), (272, 17, 2), (408, 17, 2), (374, 17, 2), (391, 17, 2), (323, 17, 2), (374, 17, 2), (374, 17, 2), (425, 17, 2)]


In [ ]:
#export
for i in range(len(shapes_2d)):
    assert shapes_2d[i][0] == shapes_3d[i][0], f'subject {i}: {shapes_2d[i][0]}, {shapes_3d[i][0]}'

Roll out all the data to lists for the generators to create batches. 

In [ ]:
#export
poses_2d_train = []
for subject in keypoints_2D.keys():
    poses_2d_train.append(keypoints_2D[subject]['custom'][0])

poses_3d_train = []
for subject in keypoints_3D.keys():
    poses_3d_train.append(keypoints_3D[subject])

assert len(poses_2d_train) == len(poses_3d_train), "Number of runs doesn't match."

poses_2d_val = []
for subject in keypoints_2D_val.keys():
    poses_2d_val.append(keypoints_2D_val[subject]['custom'][0])

poses_3d_val = []
for subject in keypoints_3D_val.keys():
    poses_3d_val.append(keypoints_3D_val[subject])

assert len(poses_2d_val) == len(poses_3d_val), "Number of runs doesn't match."

#### Define the model
> Load the temporal model trained model for generating 3D predictions.

In [ ]:
#export
# Load checkpoint
print('Loading checkpoint')
checkpoint = torch.load('pretrained_h36m_detectron_coco.bin', 
                        map_location=lambda storage,
                        loc: storage)
print('This model was trained for {} epochs'.format(checkpoint['epoch']))
checkpoint.keys()
checkpoint['model_pos']['shrink.weight'] = 

SyntaxError: invalid syntax (2572841332.py, line 9)

In [ ]:
#export
# Hyperparameters
num_joints_in = 17 # COCO
in_features = 2 # dimension of in joints
num_joints_out = 18

 # runningpose
filter_widths = [3, 3, 3, 3, 3] # just as in inference  
causal = False # No real time predictions 
dropout = 0.25 # default
channels = 1024 # default
lr = 0.001 # default
lr_decay = 0.95 # default
batch_size = 64
chunk_length = 1
num_epochs = 10

# Load two models one for training and one for evaluation
model_run_train = TemporalModel(
    num_joints_in, in_features, num_joints_out, filter_widths, causal, 
    dropout, channels
)
model_run = TemporalModel(
    num_joints_in, in_features, num_joints_out, filter_widths, causal, 
    dropout, channels
)

if torch.cuda.is_available():
    model_run_train = model_run_train.cuda()
    model_run = model_run.cuda()

# Load the pretrained model i.e to do transfer learning
model_run_train.load_state_dict(checkpoint['model_pos'])

RuntimeError: Error(s) in loading state_dict for TemporalModel:
	Missing key(s) in state_dict: "shrink.weight". 
	size mismatch for shrink.bias: copying a param with shape torch.Size([51]) from checkpoint, the shape in current model is torch.Size([54]).

In [ ]:
#export
# Calculate padding based on receptive field
receptive_field = model_run_train.receptive_field()
print('INFO: Receptive field: {} frames'.format(receptive_field))
pad = (receptive_field - 1) // 2 # Padding on each side

INFO: Receptive field: 243 frames


In [ ]:
#export
# Optimizer
optimizer = optim.Adam(model_run_train.parameters(), lr=lr, amsgrad=True)
scaler = torch.cuda.amp.GradScaler()

# Initialize loss
losses_3d_train = []
losses_3d_train_eval = []
losses_3d_valid = []

# Using batch norm momentum
initial_momentum = 0.1
final_momentum = 0.001

#### Batch generators

In [ ]:
#export
valid_generator = UnchunkedGenerator(
    cameras=None, poses_3d=poses_3d_val, poses_2d=poses_2d_val,
    pad=pad, augment=False,
    kps_left=kps_left, kps_right=kps_right, 
    joints_left=joints_left, joints_right=joints_right
)
print('INFO: Testing on {} frames'.format(valid_generator.num_frames()))
train_generator = ChunkedGenerator(
    batch_size, cameras=None, poses_3d=poses_3d_train, poses_2d=poses_2d_train, 
    pad=pad, chunk_length=chunk_length, shuffle=True, augment=False, 
    kps_left=kps_left, kps_right=kps_right, 
    joints_left=joints_left, joints_right=joints_right
)
train_generator_eval = UnchunkedGenerator(
    cameras=None, poses_3d=poses_3d_train, poses_2d=poses_2d_train, 
    pad=pad, augment=False
)
print('INFO: Training on {} frames'.format(train_generator_eval.num_frames()))

INFO: Testing on 2414 frames
INFO: Training on 6426 frames


In [ ]:
torch.cuda.is_available()

True

#### Training

In [ ]:
#export
epoch = 0
while epoch < num_epochs:
    start_time = time.time()
    # Initialize training loss
    epoch_loss_3d_train = 0
    epoch_loss_traj_train = 0
    epoch_loss_2d_train_unlabeled = 0
    N = 0
    # Regular supervised scenario
    for _, batch_3d, batch_2d in train_generator.next_epoch():
        inputs_3d = torch.from_numpy(batch_3d.astype('float32'))
        inputs_2d = torch.from_numpy(batch_2d.astype('float32'))
        if torch.cuda.is_available():
            inputs_3d = inputs_3d.cuda()
            inputs_2d = inputs_2d.cuda()
        inputs_3d[:, :, 0] = 0
        
        optimizer.zero_grad()

        # Predict 3D poses (forward) using fp16
        with torch.cuda.amp.autocast():
            predicted_3d_pos = model_run_train(inputs_2d)
            print(predicted_3d_pos[1, :, :, :])
            print(inputs_3d[1, :, :, :])
            loss_3d_pos = mpjpe(predicted_3d_pos, inputs_3d)
            N += inputs_3d.shape[0]*inputs_3d.shape[1]
            # Calculate the epoch loss
            epoch_loss_3d_train += inputs_3d.shape[0] \
            * inputs_3d.shape[1] * loss_3d_pos.item()

        # Backward
        loss_total = loss_3d_pos
        scaler.scale(loss_total).backward()
        scaler.step(optimizer)
        scaler.update()

    # Total loss over one epoch
    losses_3d_train.append(epoch_loss_3d_train / N)
    
    # End-of-epoch evaluation
    with torch.no_grad():
        # Load the newly trained network
        model_run.load_state_dict(model_run_train.state_dict())
        model_run.eval()
        # Initialize validation loss
        epoch_loss_3d_valid = 0
        epoch_loss_traj_valid = 0
        epoch_loss_2d_valid = 0
        N = 0

        # Evaluate on validation dataset
        for _, batch_3d, batch_2d in valid_generator.next_epoch():
            inputs_3d_valid = torch.from_numpy(batch_3d.astype('float32'))
            inputs_2d_valid = torch.from_numpy(batch_2d.astype('float32'))
            if torch.cuda.is_available():
                inputs_3d_valid = inputs_3d_valid.cuda()
                inputs_2d_valid = inputs_2d_valid.cuda()
            inputs_3d_valid[:, :, 0] = 0

            # Predict 3D poses (forward)
            predicted_3d_pos = model_run(inputs_2d_valid)
            loss_3d_pos = mpjpe(predicted_3d_pos, inputs_3d_valid)
            N += inputs_3d_valid.shape[0]*inputs_3d_valid.shape[1]
            epoch_loss_3d_valid += inputs_3d_valid.shape[0] \
            * inputs_3d_valid.shape[1] * loss_3d_pos.item()

        # Total loss over one epoch
        losses_3d_valid.append(epoch_loss_3d_valid / N)

        # Evaluate on training set, this time in evaluation mode
        epoch_loss_3d_train_eval = 0
        epoch_loss_traj_train_eval = 0
        epoch_loss_2d_train_labeled_eval = 0
        N = 0
        for _, batch_3d, batch_2d in train_generator_eval.next_epoch():
            if batch_2d.shape[1] == 0:
                # This can only happen when downsampling the dataset
                continue
            
            inputs_3d = torch.from_numpy(batch_3d.astype('float32'))
            inputs_2d = torch.from_numpy(batch_2d.astype('float32'))
            if torch.cuda.is_available():
                inputs_3d = inputs_3d.cuda()
                inputs_2d = inputs_2d.cuda()
            inputs_3d[:, :, 0] = 0

            # Predict 3D poses (forward)
            predicted_3d_pos = model_run(inputs_2d)
            loss_3d_pos = mpjpe(predicted_3d_pos, inputs_3d)
            N += inputs_3d.shape[0]*inputs_3d.shape[1]
            epoch_loss_3d_train_eval += inputs_3d.shape[0] \
            * inputs_3d.shape[1] * loss_3d_pos.item()

        # Total loss over one epoch
        losses_3d_train_eval.append(epoch_loss_3d_train_eval / N)

    # Calculate total training/validation time over one epoch       
    elapsed = (time.time() - start_time)/60

    print(
        f'''[{epoch+1}] time {elapsed:.2f} lr {lr} 
        3d_train {losses_3d_train[-1] * 1000} 
        3d_eval {losses_3d_train_eval[-1] * 1000} 
        3d_valid {losses_3d_valid[-1]  *1000}'''
    )

    # Decay learning rate exponentially
    lr *= lr_decay
    for param_group in optimizer.param_groups:
        param_group['lr'] *= lr_decay
    epoch += 1

    # Decay BatchNorm momentum
    momentum = initial_momentum * np.exp(
        -epoch/num_epochs * np.log(initial_momentum/final_momentum)
    )
    model_run_train.set_bn_momentum(momentum)

    # Save training curves after every epoch, as .png images
    plt.figure()
    epoch_x = np.arange(3, len(losses_3d_train)) + 1
    plt.plot(epoch_x, losses_3d_train[3:], '--', color='C0')
    plt.plot(epoch_x, losses_3d_train_eval[3:], color='C0')
    plt.plot(epoch_x, losses_3d_valid[3:], color='C1')
    plt.legend(['3d train', '3d train (eval)', '3d valid (eval)'])
    plt.ylabel('MPJPE (m)')
    plt.xlabel('Epoch')
    plt.xlim((3, epoch))
    plt.savefig('loss_plots/' + str(epoch) + '_loss_3d.png')
    plt.close('all')

tensor([[[ 0.0300,  0.0095, -0.0352],
         [-0.0993,  0.1077, -0.2664],
         [-0.1230,  0.0933, -0.0011],
         [-0.0690, -0.2869, -0.0883],
         [ 0.1113,  0.0248,  0.2059],
         [-0.1381, -0.1049,  0.0111],
         [ 0.0171, -0.0804, -0.0914],
         [-0.2942, -0.0861,  0.0732],
         [-0.0353, -0.0747, -0.2338],
         [-0.1593,  0.0959,  0.1857],
         [-0.0182, -0.0769, -0.1844],
         [-0.2834,  0.0747,  0.0365],
         [ 0.0249, -0.0602,  0.2627],
         [-0.2375, -0.2097,  0.1270],
         [ 0.0728,  0.2964,  0.1137],
         [ 0.1748, -0.2146,  0.1760],
         [ 0.0557,  0.3293, -0.3396],
         [-0.0603, -0.1461,  0.4011]]], device='cuda:0', dtype=torch.float16,
       grad_fn=<SliceBackward>)
tensor([[[     0.0000,      0.0000,      0.0000],
         [  3906.8782,   1638.6691,  -9412.5498],
         [  4000.3638,   1414.9087,  -9673.0498],
         [  3998.6448,   1493.0919,  -9313.7832],
         [  3742.1301,   1606.0068,  -9512.1

KeyboardInterrupt: 

#### Save model

In [ ]:
#export
chk_path = os.path.join('runningpose', '_epoch_{}.bin'.format(epoch))
print('Saving checkpoint to', chk_path)
torch.save({
    'epoch': epoch,
    'lr': lr,
    'random_state': train_generator.random_state(),
    'optimizer': optimizer.state_dict(),
    'model_run': model_run_train.state_dict(),
}, chk_path)

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()